# Clinical Trials Data Retrieval Pipeline

## Overview

This notebook is the **first step** in an educational pipeline demonstrating how Large Language Models (LLMs) can assist biotech and lifesciences professionals with clinical research analysis.

## Purpose

We'll retrieve, clean, and standardize clinical trial data for GLP-1 receptor agonists and related medications - a drug class that has revolutionized treatment for Type 2 Diabetes and obesity.

## Pipeline Stages

**Stage 1: Setup and Configuration**
- Import required libraries and services
- Configure logging and data storage locations

**Stage 2: Data Retrieval**
- Define target drugs to search for
- Query ClinicalTrials.gov API for relevant trials
- Save raw results for checkpoint

**Stage 3: Data Processing and Enrichment**
- Deduplicate trials that appear in multiple searches
- Apply quality filters to focus on relevant studies
- Map free-text medical conditions to standardized MeSH terms
- Clean dates and calculate trial durations
- Export processed data for downstream analysis

## Next Steps

The cleaned data from this notebook feeds into `data_annotator.ipynb`, where we'll use LLMs to extract structured insights from trial descriptions.

In [1]:
import os
import string
import time
import pickle

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
from pathlib import Path
from services import clinicaltrials_retriever, logging_config, mesh_mapper
import pandas as pd


[2025-11-13 11:14:55] INFO     - root - ================================================================================
[2025-11-13 11:14:55] INFO     - root - Logging initialized
[2025-11-13 11:14:55] INFO     - root - Log level: INFO
[2025-11-13 11:14:55] INFO     - root - Log directory: /Users/joshuaziel/Documents/Coding/glp-1_landscape/logs
[2025-11-13 11:14:55] INFO     - root - Console logging: True
[2025-11-13 11:14:55] INFO     - root - ================================================================================


In [2]:
# Load environment variables
load_dotenv(find_dotenv())

# Initialize logger
logger = logging_config.get_logger(__name__)

#Set data location
DATA_STORAGE = os.getenv("DATA_LOC", None)

if DATA_STORAGE and Path(DATA_STORAGE).exists():
    logger.info(f"Data will be saved at:{DATA_STORAGE}")
else:
    DATA_STORAGE = Path(__file__).resolve()
    logger.warning(f"Warning: Data storage path in environment does not exist or was not set, saving data here: {DATA_STORAGE}")

[2025-11-13 11:14:55] INFO     - __main__ - Data will be saved at:/Users/joshuaziel/Documents/Coding/glp-1_landscape/data


## About GLP-1 Receptor Agonists

### Background

**GLP-1 (Glucagon-Like Peptide-1)** is a hormone naturally produced in the intestine that regulates blood sugar and appetite. GLP-1 receptor agonists are medications that mimic this hormone's effects.

### Drug Categories

**Single Agonists** (Target GLP-1 receptor only)
Examples:
- Liraglutide (Victoza, Saxenda)
- Semaglutide (Ozempic, Wegovy, Rybelsus)  
- Dulaglutide (Trulicity)
- Exenatide (Byetta, Bydureon)
- Lixisenatide (Adlyxin)

**Dual Agonists** (Target GLP-1 + another receptor)
Examples:
- Tirzepatide (Mounjaro, Zepbound) - GLP-1/GIP
- Survodutide - GLP-1/Glucagon
- Cotadutide - GLP-1/Glucagon

**Triple Agonists** (Target GLP-1 + GIP + Glucagon)
Example:
- Retatrutide (in development)

### Why This Drug Class Matters

Originally developed for Type 2 Diabetes, these drugs have shown remarkable efficacy for:
- Weight management (10-20% body weight reduction)
- Cardiovascular disease prevention
- Non-alcoholic fatty liver disease (NAFLD/NASH)
- Potential neuroprotective effects

The clinical trial landscape for these drugs is rapidly evolving, making it an excellent case study for computational analysis.

### What We'll Retrieve

Below we define 17 drugs from this class based on drugs in ChEMBL. We'll search ClinicalTrials.gov for all trials testing these interventions.

In [3]:
drug_queries = {
    "EFPEGLENATIDE": "efpeglenatide",
    "LIRAGLUTIDE": "liraglutide",
    "SURVODUTIDE": "survodutide",
    "DULAGLUTIDE": "dulaglutide",
    "RETATRUTIDE": "retatrutide",
    "COTADUTIDE": "cotadutide",
    "LIXISENATIDE": "lixisenatide",
    "ALBIGLUTIDE": "albiglutide",
    "PEGSEBRENATIDE": "pegsembrenatide",
    "EXENATIDE": "exenatide",
    "PEGAPAMODUTIDE": "pegapamodutide",
    "AVEXITIDE": "avexitide",
    "TIRZEPATIDE": "tirzepatide",
    "DANUGLIPRON": "danuglipron",
    "EFINOPEGDUTIDE": "efinopegdutide",
    "TASPOGLUTIDE": "taspoglutide",
    "SEMAGLUTIDE": "semaglutide",
}

## Data Retrieval from ClinicalTrials.gov

### About ClinicalTrials.gov

**ClinicalTrials.gov** is the U.S. government's database of clinical studies conducted worldwide, maintained by the National Library of Medicine (NLM). 

### What We're Retrieving

For each drug in our list, we'll query the API to retrieve:

**Trial Metadata:**
- NCT ID (unique identifier, e.g., NCT04567890)
- Study title and brief description
- Trial phase (e.g., Phase 1, 2, 3, or 4)
- Study type (interventional vs observational)

**Enrollment Information:**
- Target enrollment numbers
- Eligibility criteria
- Inclusion of healthy volunteers

**Study Design:**
- Conditions being studied
- Interventions being tested
- Primary and secondary outcomes

**Timeline:**
- Start date and completion date
- Current recruitment status

**Sponsor Information:**
- Lead sponsor organization
- Collaborators

### Expected Duplicates

**Important:** Some trials test multiple drugs from our list (e.g., head-to-head comparisons of Semaglutide vs Liraglutide). These trials will appear in search results for both drugs. We'll handle this redundancy in the deduplication step below.

### Processing Time

Retrieving large numbers of trials may take time due to API rate limits and data parsing. You'll be able to see progress if you left LOG_TO_CONSOLE as true in your .env file.

In [4]:
drug_queries_results = []
for key in drug_queries.keys():
    drug_query = drug_queries[key]
    logger.info(f"Processing drug: {key} with query term: {drug_query}")

    # Retrieve clinical trials
    results = clinicaltrials_retriever.retrieve_batched_studies(query_intervention=drug_query)
    logger.info(f"Retrieved {results.shape[0]} clinical trials for drug: {key}")
    results['drug_name'] = key
    drug_queries_results.append(results)

all_trials = pd.concat(drug_queries_results, axis = 0, ignore_index=True)
all_trials.to_csv(f"{DATA_STORAGE}/all_raw_trials.csv")
logger.info(f"Retrieved a total of {all_trials.shape[0]} and saved to csv:{DATA_STORAGE}/all_raw_trials.csv")


[2025-11-13 11:14:55] INFO     - __main__ - Processing drug: EFPEGLENATIDE with query term: efpeglenatide
[2025-11-13 11:14:55] INFO     - services.clinicaltrials_retriever - Starting ClinicalTrials.gov study retrieval
[2025-11-13 11:14:55] INFO     - services.clinicaltrials_retriever - Starting ClinicalTrials.gov study retrieval
[2025-11-13 11:14:55] INFO     - utils.clinicaltrials - Retrieving study details from ClinicalTrials.gov
[2025-11-13 11:14:56] INFO     - utils.clinicaltrials - Total studies available: 5
[2025-11-13 11:14:56] INFO     - utils.clinicaltrials - No more pages available
[2025-11-13 11:14:56] INFO     - utils.clinicaltrials - Successfully retrieved 5 studies from ClinicalTrials.gov
[2025-11-13 11:14:56] INFO     - services.clinicaltrials_retriever - Successfully processed 5 studies
[2025-11-13 11:14:56] INFO     - __main__ - Retrieved 5 clinical trials for drug: EFPEGLENATIDE
[2025-11-13 11:14:56] INFO     - __main__ - Processing drug: LIRAGLUTIDE with query term:

## Data Quality and Standardization

Raw clinical trial data from ClinicalTrials.gov requires significant cleaning before meaningful analysis. This section performs several critical transformations:

### 1. Deduplication

**The Problem:**  
Trials testing multiple drugs from our list appear multiple times in our raw results - once for each drug query.

**The Solution:**  
We identify duplicate trials by their unique **NCT ID** (ClinicalTrials.gov identifier). For each duplicate:
- Keep a single record
- Combine all associated drug names into a list
- Preserve all other trial metadata

**Example:**
- Raw data: 2 rows for NCT12345678 (one tagged "SEMAGLUTIDE", one tagged "LIRAGLUTIDE")  
- Cleaned data: 1 row for NCT12345678 with `drug_name = ["SEMAGLUTIDE", "LIRAGLUTIDE"]`

This approach maintains complete information while eliminating redundancy.

### 2. Quality Filtering

Not all trials in ClinicalTrials.gov are relevant for our drug efficacy analysis. We apply strict filters to focus on high-quality, patient-focused interventional studies.

**Filters Applied:**

✅ **Drug/Biological Interventions Only**  
- Include: Pharmaceutical drugs and biologics
- Exclude: Behavioral interventions, devices, surgical procedures, dietary supplements

✅ **Patient Populations (No Healthy Volunteers)**  
- Include: Studies enrolling patients with medical conditions
- Exclude: Pharmacokinetic studies in healthy volunteers (Phase 1 safety studies)
- *Why?* We're interested in therapeutic efficacy, not basic safety/metabolism studies

✅ **Interventional Studies Only**  
- Include: Randomized controlled trials (RCTs), single-arm trials where interventions are assigned
- Exclude: Observational/epidemiological studies
- *Why?* Interventional studies provide the strongest evidence for drug efficacy

✅ **Well-Defined Endpoints**  
- Include: Trials with documented primary outcome measures
- Exclude: Trials missing outcome definitions
- *Why?* Ensures we can meaningfully analyze what the trial is measuring

✅ **Documented Interventions**  
- Include: Trials with clear intervention descriptions
- Exclude: Trials with missing or empty intervention fields
- *Why?* Essential metadata for understanding study design

**Expected Impact:**  
These filters typically remove 20-30% of raw trials, focusing our analysis on the most clinically relevant studies.

### 3. Medical Terminology Standardization with MeSH

#### The Challenge: Inconsistent Medical Terminology

Clinical trials describe conditions using varied, non-standardized terminology. For example, these all refer to the same condition:

| Free-Text in ClinicalTrials.gov | All Mean... |
|--------------------------------|-------------|
| "Type 2 Diabetes Mellitus" | Same condition |
| "T2DM" | Same condition |  
| "Diabetes Mellitus, Type 2" | Same condition |
| "Non-Insulin-Dependent Diabetes" | Same condition |
| "Adult-Onset Diabetes" | Same condition |

Without standardization, analysis becomes impossible:
- Counting trials by condition gives fragmented results
- Text-based grouping misses synonyms and abbreviations  
- Comparison across datasets requires manual mapping

#### The Solution: MeSH (Medical Subject Headings)

**MeSH** is the National Library of Medicine's controlled vocabulary for biomedical concepts, used to index all PubMed articles since 1960.

**Key Features:**
- **Controlled vocabulary**: Single canonical term per concept
- **Hierarchical structure**: Terms organized in tree-like classifications (e.g., Diseases → Metabolic Diseases → Diabetes Mellitus)
- **Unique identifiers**: Each term has a stable MeSH ID (e.g., D003924)
- **Mapping service**: APIs to map free-text to official MeSH terms

#### Our MeSH Mapping Process

1. **Extract unique conditions** from all filtered trials
2. **Query NCBI's E-utilities API** for each condition
3. **Filter to disease/disorder terms only** (MeSH tree codes starting with 'C' or 'F')
4. **Return the best match**: Standardized MeSH term + unique ID
5. **Apply mappings** to create a new `matched_conditions` column
6. **Note** this tool is pretty brittle - we expect it to not find a match much of the time, which is something we'll address in the next state

#### Example Mapping Result

```
Input:  "Type II Diabetes"
Output: "Diabetes Mellitus, Type 2 (MeSH ID:D003924)"
```

#### Why This Matters

- **Accurate grouping**: All trials studying diabetes map to the same MeSH term
- **Literature integration**: Can link to PubMed articles using same MeSH tags
- **Downstream analysis**: LLMs and algorithms can work with consistent terminology
- **Reproducibility**: MeSH terms are stable over time (unlike free-text)

#### Processing Time

This step takes several minutes because:
- Each unique condition requires an API call to NCBI  
- We respect rate limits (0.35 seconds between requests = ~3 requests/second)
- Typical dataset has 100-200 unique condition terms

**Progress bar below shows real-time mapping status.**

### 4. Additional Data Cleaning and Enrichment

Before exporting, we perform several final transformations to make the data more analysis-ready:

#### Date Standardization
Clinical trial dates come in various formats:
- `"2023"` (year only)
- `"2023-06"` (year-month)
- `"2023-06-15"` (full date)

We convert all dates to Python `datetime` objects for consistent processing.

#### Trial Duration Calculation  
Using standardized dates, we calculate trial duration in years:
```
duration = (completion_date - start_date) / 365.25
```

This metric helps identify:
- Short-term efficacy studies (0.5-2 years)
- Long-term safety studies (3+ years)
- Stalled or abandoned trials (unrealistic durations)

#### Sponsor Name Cleaning
Pharmaceutical company names appear with inconsistent formatting:
- "Novo Nordisk A/S"
- "NOVO NORDISK"  
- "Novo-Nordisk"

We remove punctuation and standardize capitalization for better grouping in analysis.

#### New Column: `llm_annotations`
We initialize an empty list for each trial. This will be populated in the next notebook (`data_annotator.ipynb`) where we'll use LLMs to extract structured insights from trial descriptions.

---

### 5. Export Formats

We save the cleaned dataset in **two formats**, each optimized for different use cases:

#### CSV Format (`cleaned_trials.csv`)

**Pros:**
- Human-readable in text editors
- Opens in Excel, Google Sheets, or any spreadsheet software
- Easy to share with non-technical stakeholders
- Version control friendly (can see line-by-line diffs)

**Cons:**
- Lists and complex data types get flattened to strings
- Datetime objects converted to text
- Larger file size
- Slower to load for large datasets

**Best for:** Quick review, sharing, manual inspection

---

#### Pickle Format (`cleaned_trials.pkl`)

**Pros:**
- Preserves Python data types exactly:
  - Lists stay as lists (e.g., `drug_name`, `conditions`)
  - Datetime objects stay as datetime objects
  - Complex nested structures maintained
- Faster to load and save
- Smaller file size (binary compression)

**Cons:**
- Not human-readable (binary format)
- Python-specific (can't open in Excel)
- Version control shows binary diffs (not useful for code review)

**Best for:** Downstream Python analysis

---

### ⚠️ Important for Next Steps

**Use the pickle file (`cleaned_trials.pkl`) in the next notebook (`data_annotator.ipynb`).**

Why? The pickle preserves list structures and datetime objects that are essential for LLM annotation workflows.

---

### Summary of New/Cleaned Columns

| Column | Description | Example |
|--------|-------------|---------|
| `drug_name` | List of GLP-1 drugs tested | `["SEMAGLUTIDE", "LIRAGLUTIDE"]` |
| `cleaned_sponsor` | Standardized sponsor name | `"NOVO NORDISK"` |
| `cln_start_date` | Parsed start date | `datetime(2023, 6, 1)` |
| `cln_completion_date` | Parsed completion date | `datetime(2025, 12, 31)` |
| `duration` | Trial length in years | `2.6` |
| `matched_conditions` | MeSH-standardized conditions | `["Diabetes Mellitus, Type 2 (MeSH ID:D003924)"]` |
| `llm_annotations` | Placeholder to trace data added by LLMs| `[]` (empty, filled in next notebook) |

In [5]:
def deduplicate_trials(df):
    """Deduplicate trials and combine drug names"""
    
    # Pre-allocate list to collect results
    deduped_list = []
    
    # Group by nct_id to handle duplicates
    for nct_id, group in df.groupby('nct_id'):
        if len(group) > 1:
            logger.info(f"Deduplicating {len(group)} entries for NCT ID: {nct_id}")
        
        # Take first row as template and ensure drug_name column can hold lists
        template = group.iloc[0].copy()
        
        # Get all unique drug names for this trial
        drug_names = group['drug_name'].dropna().unique().tolist()
        
        # Convert to Series to allow assignment
        template = pd.Series(template)
        template['drug_name'] = drug_names
        
        deduped_list.append(template)
    
    # Create DataFrame from list (much more efficient)
    deduped_results = pd.DataFrame(deduped_list)
    
    return deduped_results

def remove_punctuation(text):
    """Remove punctuation with null handling"""
    if pd.isna(text) or text is None:
        return ""
    return str(text).translate(str.maketrans('', '', string.punctuation)).upper()

def convert_date(text):
    """
    Convert date to standard object
    """
    if pd.isna(text) or text is None:
        return None
    elif "-" in text:
        date_info = text.split("-")
        if len(date_info) == 2:
            year = int(date_info[0])
            month = int(date_info[1])
            return datetime(year = year, month = month, day = 1)
        if len(date_info) == 3:
            year = int(date_info[0])
            month = int(date_info[1])
            day = int(date_info[2])
            return datetime(year = year, month = month, day = day)
    else:
        return None

def calculate_duration(row):
    """
    Precise calculation using total_seconds with robust null handling
    """
    start_date = row['cln_start_date']
    completion_date = row['cln_completion_date']
    
    # Handle various types of missing/empty values
    if (pd.isna(start_date) or pd.isna(completion_date) or 
        start_date is None or completion_date is None or
        start_date == '' or completion_date == ''):
        return None
    
    # Calculate duration using total_seconds for precision
    duration = completion_date - start_date
    seconds_per_year = 365.25 * 24 * 60 * 60
    years = duration.total_seconds() / seconds_per_year
    return round(years, 1)

def generate_mesh_term_map(df):
    """
    Map conditions column to best matching MeSH condition term
    """
    unique_conditions = df.conditions.explode().unique()
    term_mappings = {}
    for condition in tqdm(unique_conditions, desc="Matching conditions to MeSH terms"):
        if pd.notna(condition) and condition:  # Skip null/empty conditions
            result = mesh_mapper.search_mesh_term(condition, filter_diseases_only=True)  # Change to True to filter
            if result:
                mapping = f"{result["mesh_term"]} (MeSH ID:{result["mesh_id"]})"
                term_mappings[condition] = mapping
            # Add delay after each search to respect rate limits
            time.sleep(0.35)
    # Display results
    logger.info(f"\nSuccessfully matched {len(term_mappings.keys())} conditions to MeSH terms")
    return term_mappings

def add_mesh_mappings(conditions, term_map):
    """
    Apply the MeSH term mapping to the Data
    """
    matched_conditions = []
    if conditions is not None and len(conditions)>0:  # Skip null/empty conditions
        for condition in conditions:
            if term_map.keys() and condition in term_map.keys():
                matched_term = term_map[condition]
                logger.info(f"Found a MeSH term mapping for {condition}")
                matched_conditions.append(matched_term)
            else:
                logger.info(f"No existing MeSH term mapping for {condition}")
    return list(set(matched_conditions))

#Deduplication
logger.info(f"Deduplicating {all_trials.shape[0]} trials")
deduped_trials = deduplicate_trials(all_trials)
logger.info(f"Removed {all_trials.shape[0]-deduped_trials.shape[0]} trials during de-duplication")

#Data Cleaning
logger.info(f"Initiating data clean-up on {deduped_trials.shape[0]} trials.")
cleaned_trials = deduped_trials[
    # Check if 'DRUG' is in intervention_types list
    (deduped_trials['intervention_types'].apply(
        lambda x: isinstance(x, list) and ('DRUG' or 'BIOLOGICAL') in x
    )) 
    & 
    # Check healthy_volunteers 
    ((deduped_trials['healthy_volunteers'] == False) | 
     (deduped_trials['healthy_volunteers'].isnull()))
    & 
    # Check study type
    (deduped_trials['study_type'] == 'INTERVENTIONAL')
    &
    # Check if primary_outcomes is not empty/null
    (deduped_trials['primary_outcomes'].notna() & 
     (deduped_trials['primary_outcomes'].astype(str) != '[]'))
    &
    # Check if interventions is not empty/null  
    (deduped_trials['interventions'].notna() & 
     (deduped_trials['interventions'].astype(str) != '[]'))
].copy()
mesh_term_map = generate_mesh_term_map(cleaned_trials)
mesh_frame = pd.DataFrame.from_dict(mesh_term_map, orient = 'index')
mesh_frame.to_csv(f"{DATA_STORAGE}/mesh_term_mappings.csv")


cleaned_trials['cleaned_sponsor'] = cleaned_trials['lead_sponsor'].apply(remove_punctuation)
cleaned_trials['cln_start_date']= cleaned_trials['start_date'].apply(convert_date)
cleaned_trials['cln_completion_date']= cleaned_trials['completion_date'].apply(convert_date)
cleaned_trials['duration'] = cleaned_trials.apply(calculate_duration, axis = 1)
cleaned_trials["matched_conditions"] = cleaned_trials["conditions"].apply(add_mesh_mappings, term_map=mesh_term_map)
cleaned_trials['llm_annotations'] = [[] for _ in range(len(cleaned_trials))]
cleaned_trials.shape

#Trial saved as CSV for ease of review and as a pkl - the pkl should be used by the annotator notebook
cleaned_trials.to_csv(f"{DATA_STORAGE}/cleaned_trials.csv")
with open(f"{DATA_STORAGE}/cleaned_trials.pkl", 'wb') as f:
    pickle.dump(cleaned_trials, f)
logger.info(f"Saved {cleaned_trials.shape[0]} trials after data cleaning in {DATA_STORAGE}")

[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2061 trials
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT00518115
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT00518882
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT00696657
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT00707031
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT00717457
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT01029886
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT01064687
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT01128894
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT01175473
[2025-11-13 11:15:17] INFO     - __main__ - Deduplicating 2 entries for NCT ID: NCT01181986
[2025-11-1

In [6]:
##Uncomment to reload the Dataframe from pickle of Necessary
#cleaned_trials_loc = "data/cleaned_trials.pkl"
#if Path(cleaned_trials_loc).exists():
#    with open(cleaned_trials_loc, "rb") as f:
#        cleaned_trials = pickle.load(f)